In [ ]:
# imports
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as sig
import simpleaudio as sa

In [2]:
# functions developed in the lab

def timescale(x, fs, a):
    #n, d = decimal.Decimal(a).as_integer_ratio()
    [n, d] = (np.double(a)).as_integer_ratio()
    y = sig.resample_poly(x,d,n)
    t = np.arange(0,len(y),1)*(1.0/fs)
    return y,t

def play_audio(file_name):
    fs, audio_data = wav.read(file_name)
    play_obj = sa.play_buffer(audio_data, audio_data.ndim, 2, fs)
    play_obj.wait_done()

def timeshift(x, fs, t0):
    n0 = int(fs * t0)
    
    if t0 > 0:  # time delay
        y = np.concatenate((np.zeros(n0), x))
    else:  # time advance
        y = np.concatenate((x[-n0:], np.zeros(-n0)))
    
    t = np.arange(0, len(y), 1) * (1.0 / fs)
    return y, t


In [7]:
def normalize_audio(audio):
    audio = audio.astype(np.float32)
    audio /= np.max(np.abs(audio))
    return audio

# Load two sounds (either provided, from an open repository, or your own collection)
fs1, x1 = wav.read('tuba11.wav')
fs2, x2 = wav.read('flute11.wav')

# If the sounds have 2 channels, extract audio data from only one channel
if x1.ndim > 1:
    x1 = x1[:, 0]
if x2.ndim > 1:
    x2 = x2[:, 0]

# Normalize the audio signals
x1 = normalize_audio(x1)
x2 = normalize_audio(x2)

# Time scaling
a = 0.5
scaled_x1, _ = timescale(x1, fs1, a)

# Time shifting
shifted_x2, _ = timeshift(x2, fs2, 1.0)  # Shift by 1 second

# Pad the shorter signal with zeros to make them the same length
len_diff = abs(len(scaled_x1) - len(shifted_x2))
if len(scaled_x1) > len(shifted_x2):
    shifted_x2 = np.concatenate((shifted_x2, np.zeros(len_diff)))
else:
    scaled_x1 = np.concatenate((scaled_x1, np.zeros(len_diff)))

# Addition (scale the result by 0.5 to avoid clipping)
added_signals = 0.5 * (scaled_x1 + shifted_x2)

# Multiplication
multiplied_signals = scaled_x1 * shifted_x2

# Concatenate sounds with a brief silence (vector of zeros)
silence = np.zeros(int(fs1 * 0.5))  # 0.5-second silence
concatenated_signals = np.concatenate((multiplied_signals, silence, added_signals))

# Save the new signal to a WAV file (no more than 20 seconds in length)
wav.write('new_signal.wav', fs1, (concatenated_signals * 32767).astype(np.int16))

# Play the new signal
play_audio('new_signal.wav')
